In [1]:
cd ..

/Users/olivierbougeant/prodigyfinance/projects/21_conversion_rate_synthetic_control/synthetic_control


/Users/olivierbougeant/prodigyfinance/venv/synthetic_control/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from datetime import datetime
import pandas as pd
from synthetic_control import SyntheticControl

# Read California proposition 99 dataset

In [3]:
df = pd.read_csv("./examples/california_prop99.csv")

In [4]:
df["Year"] = pd.to_datetime(df["Year"], format='%Y')

In [5]:
df = df.pivot(index="Year", columns="State",values="PacksPerCapita")

In [6]:
df.head()

State,Alabama,Arkansas,California,Colorado,Connecticut,Delaware,Georgia,Idaho,Illinois,Indiana,...,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1970-01-01,89.800003,100.300003,123.000000,124.800003,120.000000,155.000000,109.900002,102.400002,124.800003,134.600006,...,103.599998,92.699997,99.800003,106.400002,65.500000,122.599998,124.300003,114.500000,106.400002,132.199997
1971-01-01,95.400002,104.099998,121.000000,125.500000,117.599998,161.100006,115.699997,108.500000,125.599998,139.300003,...,115.000000,96.699997,106.300003,108.900002,67.699997,124.400002,128.399994,111.500000,105.400002,131.699997
1972-01-01,101.099998,103.900002,123.500000,134.300003,110.800003,156.300003,117.000000,126.099998,126.599998,149.199997,...,118.699997,103.000000,111.500000,108.599998,71.300003,138.000000,137.000000,117.500000,108.800003,140.000000
1973-01-01,102.900002,108.000000,124.400002,137.899994,109.300003,154.699997,119.800003,121.800003,124.400002,156.000000,...,125.500000,103.500000,109.699997,110.400002,72.699997,146.800003,143.100006,116.599998,109.500000,141.199997
1974-01-01,108.199997,109.699997,126.699997,132.800003,112.400002,151.300003,123.699997,125.599998,131.899994,159.600006,...,129.699997,108.400002,114.800003,114.699997,75.599998,151.800003,149.600006,119.900002,111.800003,145.800003


In [7]:
X = df.drop(columns="California")
y = df["California"]

# Generate predictions for the Synthetic Control group

In [8]:
sc = SyntheticControl(treatment_start=datetime(year=1989, month=1, day=1))

In [9]:
y_synth = sc.fit_predict(X, y)

In [10]:
y_ci = sc.get_confidence_interval(X, y)

In [11]:
y_ci.head()

,5,10,20,30,40,50,60,70,80,90,95
Year,,,,,,,,,,,
1970-01-01,91.282443,94.042580,98.237214,99.703054,100.592195,103.187232,106.352095,110.131863,113.668897,118.099523,119.188477
1971-01-01,95.856289,96.266624,101.180542,102.773978,105.034041,107.623240,109.646319,110.758018,116.187789,120.327445,121.473699
1972-01-01,99.686277,100.983268,102.490851,105.502366,108.969568,113.815524,118.516587,120.399000,122.515762,126.498340,128.558836
1973-01-01,102.342662,104.223019,105.645804,107.116398,110.611227,113.369194,118.846349,121.706114,123.705712,125.824220,127.483143
1974-01-01,106.623464,107.534218,108.278568,110.622431,114.225262,114.982395,117.200519,120.007048,124.017490,126.702653,128.751026


# Display predictions for the Synthetic Control group

In [12]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

In [13]:
y_synth

array([118.79500632, 122.28449874, 127.60450826, 125.93954562,
       125.74886499, 126.00264499, 130.3594663 , 127.18658768,
       125.05205004, 121.29515995, 119.14728402, 118.25096397,
       115.76062695, 111.60937619, 104.11845813, 101.7857342 ,
        99.11833624,  97.90294358,  92.73202874,  89.41045432,
        84.26021441,  80.23078836,  79.10987468,  79.57229186,
        79.00677051,  79.73631645,  78.77939848,  80.44393248,
        81.06456658,  79.61785942,  72.59992477])

In [20]:
plot_data = [
    go.Scatter(x=y.index, y=y),
    go.Scatter(x=y.index, y=y_synth),
    # go.Scatter(x=time, y=quantile_synth_data[0.5], line_color=DEFAULT_PLOTLY_COLORS[1], name="Synthetic Control Group"),
    # go.Scatter(x=time, y=quantile_synth_data[0.4], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=True, name="90% Confidence Interval", legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.6], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.3], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.7], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.2], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.8], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.1], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.9], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.05], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=time, y=quantile_synth_data[0.95], line_color=opacity_color, fill="tonexty", fillcolor=opacity_color, showlegend=False, legendgroup="CI"),
    # go.Scatter(x=[promotion_date] *2, y=[0, 1], mode="lines", line={"color": "black", "dash": "dash"}, name="Promo start"),
    # go.Scatter(x=[promo_end_date] *2, y=[0, 1], mode="lines", line={"color": "black", "dash": "dot"}, name="Promo end")
]

In [ ]:
go.Layout()

In [21]:
go.Figure(plot_data)